In [1]:
import numpy as np
import os
import time
import torch
import torch.nn as nn
import pickle
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from diffusers.schedulers.scheduling_ddpm import DDPMScheduler
from diffusers.training_utils import EMAModel
from diffusers.optimization import get_scheduler
from tqdm.auto import tqdm
import submodules.data_filter as _df
import diffusion_pipline.data_processing as dproc
import diffusion_pipline.model as md
import submodules.cleaned_file_parser as cfp


In [2]:
checkpoint_path = '/home1/shuklar/diff_files/checkpoints/checkpoint_3BODY_12_markers_edge_1_step_all_no_battery_epoch_199.pth' #TODO

checkpoint = torch.load(checkpoint_path)

In [3]:
checkpoint.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'ema_state_dict', 'len_dataloader', 'dataset_stats', 'num_epochs', 'obs_dim', 'action_dim', 'pred_horizon', 'obs_horizon', 'action_horizon', 'target_fps', 'action_item', 'obs_item', 'marker_item', 'num_diffusion_iters'])

In [4]:
checkpoint['marker_item']

{'A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'C1', 'C2', 'D1', 'D2', 'D3', 'D4'}

In [5]:
# Parameters corrsponding to
save_dir = "/home1/shuklar/diff_files/trun_table_task/pred_csvs/" #TODO
test_base_dir = "/home1/shuklar/diff_files/trun_table_task/edge_test/" #TODO

num_epochs =checkpoint['num_epochs']
obs_dim = checkpoint['obs_dim']
action_dim = checkpoint['action_dim']
# parameters
pred_horizon = checkpoint['pred_horizon']
obs_horizon = checkpoint['obs_horizon']
action_horizon = checkpoint['action_horizon']
target_fps = checkpoint['target_fps']

action_item = checkpoint['action_item']
obs_item = checkpoint['obs_item']


# create network object
noise_pred_net = md.ConditionalUnet1D(
    input_dim=action_dim,
    global_cond_dim=obs_dim*obs_horizon
)

# example inputs
noised_action = torch.randn((1, pred_horizon, action_dim))
obs = torch.zeros((1, obs_horizon, obs_dim))
diffusion_iter = torch.zeros((1,))

# the noise prediction network
# takes noisy action, diffusion iteration and observation as input
# predicts the noise added to action
noise = noise_pred_net(
    sample=noised_action,
    timestep=diffusion_iter,
    global_cond=obs.flatten(start_dim=1))

# illustration of removing noise
# the actual noise removal is performed by NoiseScheduler
# and is dependent on the diffusion noise schedule
denoised_action = noised_action - noise

# for this demo, we use DDPMScheduler with 100 diffusion iterations
num_diffusion_iters = checkpoint['num_diffusion_iters']
noise_scheduler = DDPMScheduler(
    num_train_timesteps=num_diffusion_iters,
    # the choise of beta schedule has big impact on performance
    # we found squared cosine works the best
    beta_schedule='squaredcos_cap_v2',
    # clip output to [-1,1] to improve stability
    clip_sample=True,
    # our network predicts noise (instead of denoised action)
    prediction_type='epsilon'
)

# device transfer
device = torch.device('cuda')
_ = noise_pred_net.to(device)

# Exponential Moving Average
# accelerates training and improves stability
# holds a copy of the model weights
ema = EMAModel(
    parameters=noise_pred_net.parameters(),
    power=0.75)

# Standard ADAM optimizer
# Note that EMA parametesr are not optimized
optimizer = torch.optim.AdamW(
    params=noise_pred_net.parameters(),
    lr=1e-4, weight_decay=1e-6)

# Cosine LR schedule with linear warmup
lr_scheduler = get_scheduler(
    name='cosine',
    optimizer=optimizer,
    num_warmup_steps=200,
    num_training_steps=checkpoint['len_dataloader'] * num_epochs
)

ema_noise_pred_net = noise_pred_net

number of parameters: 6.677608e+07


In [6]:
noise_pred_net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
lr_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
ema.load_state_dict(checkpoint['ema_state_dict'])
start_epoch = checkpoint['epoch'] + 1

In [7]:
action_item

['chisel', 'gripper']

In [8]:
obs_item

['battery']

In [9]:
# Load data
dict_of_df_rigid_test = {}
dict_of_df_marker_test = {}
name = []

# for file in os.listdir(test_base_dir):
#     name.append(file)
for file in os.listdir(test_base_dir):
    if file.endswith(".csv"):
        name.append(file)
        path_name = test_base_dir + file
        data_test = cfp.DataParser.from_quat_file(file_path = path_name, target_fps=target_fps, filter=True, window_size=15, polyorder=3)

        marker_data = data_test.get_marker_Txyz() #UNCOM
        
        # filtered_marker_data = data_test.get_marker_Txyz() #COM
        
        # List of keys you want to extract
        # keys = ['A1', 'A2', 'A3', 'A4'] #COM
        
        # Using dictionary comprehension to get only the specified keys
        # marker_data = {key: filtered_marker_data[key] for key in keys} #COM
        data_state_dict = data_test.get_rigid_TxyzRxyz()

        dicts = [data_state_dict, marker_data]
        trimmed_dicts = _df.trim_lists_in_dicts(dicts)

        
        dict_of_df_rigid_test[file] = trimmed_dicts[0]
        dict_of_df_marker_test[file] = trimmed_dicts[1]


item_name_test = data_test.rigid_bodies
# marker_name_test = ['A1', 'A2', 'A3', 'A4'] #COM
marker_name_test = data_test.markers # UNCOM


if len(dict_of_df_rigid_test) == len(dict_of_df_marker_test):

    rigiddataset_test, index_test = _df.episode_combiner(dict_of_df_rigid_test, item_name_test)
    markerdataset_test, _ = _df.episode_combiner(dict_of_df_marker_test, marker_name_test)

indexes = index_test[action_item[0]]
action = []
obs = []
for i in range(indexes[-1]):
    # a = []
    a = np.concatenate([rigiddataset_test[item][i] for item in action_item])
    # print(a)

    b = np.concatenate([rigiddataset_test[item][i] for item in action_item] + [rigiddataset_test[item][i] for item in obs_item] + [markerdataset_test[item][i] for item in marker_name_test])
    # print(b)
    
    action.append(a)
    obs.append(b)
    
# All demonstration episodes are concatinated in the first dimension N
action = np.array(action, dtype=np.float64)
obs = np.array(obs, dtype=np.float64)

# Initialize lists to store segmented data
splits_obs = []
splits_action = []
previous_index = 0

# Iterate through episode_ends and slice action and obs accordingly
for index in indexes:
    splits_obs.append(obs[previous_index:index + 1])  # Include index itself in the slice
    splits_action.append(action[previous_index:index + 1])
    previous_index = index + 1

In [10]:
len(splits_obs[0][0])

54

In [11]:
len(splits_action[0][0])

12

In [12]:
marker_name_test

{'A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'C1', 'C2', 'D1', 'D2', 'D3', 'D4'}

In [13]:
len(splits_action[0])

191

In [29]:
import collections

trajectories = {}
losses_per_traj = {}
for j in range(len(indexes)):
    # print(j)
    # get first observation
    com_obs = splits_obs[j]
    obs = splits_obs[j][0]
    actions_test = splits_action[j]
    # max_steps = len(actions_test) # TODO -for matching total prediction horizon
    max_steps = 500 #TODO - for 500 prdiction horizon
    stats = checkpoint['dataset_stats']
    # keep a queue of last 2 steps of observations
    obs_deque = collections.deque(
        [obs] * obs_horizon, maxlen=obs_horizon)

    # save visualization and rewards
    done = False
    step_idx = 0
    traj = []
    loss_com = []
    with tqdm(total=max_steps, desc="Eval") as pbar:
        while not done:
            B = 1
            # stack the last obs_horizon (2) number of observations
            obs_seq = np.stack(obs_deque)
            # print("Obs_sep -",obs_seq)
            # normalize observation
            nobs = dproc.normalize_data(obs_seq, stats=stats['obs'])
            # print(nobs)
            # device transfer
            nobs = torch.from_numpy(nobs).to(device, dtype=torch.float32)
            # infer action
            with torch.no_grad():
                # reshape observation to (B,obs_horizon*obs_dim)
                obs_cond = nobs.unsqueeze(0).flatten(start_dim=1)
                # print(obs_cond.shape)

                # initialize action from Guassian noise
                noisy_action = torch.randn(
                    (B, pred_horizon, action_dim), device=device)
                naction = noisy_action

                # init scheduler
                noise_scheduler.set_timesteps(num_diffusion_iters)

                for k in noise_scheduler.timesteps:
                    # predict noise
                    noise_pred = ema_noise_pred_net(
                        sample=naction,
                        timestep=k,
                        global_cond=obs_cond
                    )

                    # inverse diffusion step (remove noise)
                    naction = noise_scheduler.step(
                        model_output=noise_pred,
                        timestep=k,
                        sample=naction
                    ).prev_sample

            # unnormalize action
            naction = naction.detach().to('cpu').numpy()
            # (B, pred_horizon, action_dim)
            # print(len(naction[0]))
            naction = naction[0]
            action_pred = dproc.unnormalize_data(naction, stats=stats['action'])

            # only take action_horizon number of actions
            start = obs_horizon - 1
            end = start + action_horizon
            action = action_pred[start:end,:]
            # traj.extend(action)
            losses = []
                
            for i in range(len(action)):
            # loss
            # print("Action_pred -",action[0])
            # print("Action_orignal -",actions_test[0])
            # # print("Obs_added without pred-",com_obs[i])
                if len(action) > len(actions_test):
                    done = True
                if done:
                    break
                # loss_test = nn.functional.mse_loss(torch.tensor(action[i]), torch.tensor(actions_test[i])) # TODO - for original prediction horizon
                action_last = list(action[i])
                # print("Action_last ---",action_last)
                
                if step_idx < max_steps - len(actions_test):
                    com_obs_part = list(obs[len(action_last):])
                    obs_deque.append(action_last + com_obs_part)

                else:
                    com_obs_part = list(com_obs[i][len(action_last):])
                    # print("Obs to add", com_obs_part)
                    # Concatenating prediction to the obs lists
                    com_obs[i] = action_last + com_obs_part
                    # print("Obs_added with pred -",com_obs[i])
                    obs_deque.append(com_obs[i])


                
                ############################################
                #### original pred horizon ################
                ############################################
                # com_obs_part = list(com_obs[i][len(action_last):])
                # # print("Obs to add", com_obs_part)
                # # Concatenating prediction to the obs lists
                # com_obs[i] = action_last + com_obs_part
                # # print("Obs_added with pred -",com_obs[i])
                # obs_deque.append(com_obs[i])
                ############################################

                
                # losses.append(loss_test.item())
                # update progress bar
                step_idx += 1
                pbar.update(1)
                # pbar.set_postfix(loss=np.mean(losses))
                # print(i)
                traj.append(action_last + com_obs_part)
                if step_idx > max_steps:
                    done = True
                if done:
                    break

            if step_idx > max_steps - len(actions_test):
                com_obs = com_obs[len(action):]
                actions_test = actions_test[len(action):]
            # com_obs = com_obs[1:]
            # actions_test = actions_test[1:]
            # loss_com.append(np.mean(losses).tolist())
    # losses_per_traj[f"{name[j]}"] = np.nanmean(loss_com)
    trajectories[f"{name[j]}"] = traj

Eval:  99%|█████████▉| 496/500 [00:34<00:00, 14.32it/s]


In [31]:
len(trajectories['ft_450_edge_1_step_1.csv'][0])

54

In [32]:
import csv

save_dir = "/home1/shuklar/diff_files/trun_table_task/pred_csvs/"
for index, key in enumerate(trajectories.keys()):
    print(index)
    # Define the file path or name
    file_path = f'{save_dir}/pred_{key}'

    save_type='EULER'
    # add first rows
    _params = {
        'QUAT': {'len':7,
                    'dof': ['X', 'Y', 'Z', 'w', 'x', 'y', 'z']},
        'EULER': {'len':6,
                    'dof': ['X', 'Y', 'Z', 'x', 'y', 'z']},
        'Vel': {'len':6,
                    'dof': ['Xv', 'Yv', 'Zv', 'xv', 'yv', 'zv']}
    }
    
    _SUP_HEADER_ROW = (["RigidBody"] *  len(data_test.rigid_bodies) * _params[save_type]['len'] + ["Marker"] * len(marker_name_test) * 3)
    _FPS_ROW = ["FPS", target_fps] + [0.0]*(len(_SUP_HEADER_ROW) - 2)
    _rb_col_names = [f"{rb}_{axis}" for rb in action_item for axis in _params[save_type]['dof']]
    _obs_col_name = [f"{rb}_{axis}" for rb in obs_item for axis in _params[save_type]['dof']]
    _mk_col_names = [f"{mk}_{axis}" for mk in marker_name_test for axis in ['X', 'Y', 'Z']]
    _HEADER_ROW = _rb_col_names + _obs_col_name + _mk_col_names
        # Open the file in write mode
    with open(file_path, 'w') as file:
        writer = csv.writer(file)
        writer.writerow(_SUP_HEADER_ROW)
        writer.writerow(_FPS_ROW)
        writer.writerow(_HEADER_ROW)
        writer.writerows(trajectories[key])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
